In [4]:
import os

def label_images(folder_path, prefix):
    # Get a list of all files in the folder
    files = os.listdir(folder_path)
    
    # Initialize a counter
    counter = 1
    
    for filename in files:
        # Ensure we only process image files
        if filename.endswith('.png') or filename.endswith('.jpg') or filename.endswith('.jpeg'):
            # Create the new filename
            new_filename = f"{prefix}{counter}.png"  # or use .jpg depending on your files
            old_file_path = os.path.join(folder_path, filename)
            new_file_path = os.path.join(folder_path, new_filename)
            
            # Rename the file
            os.rename(old_file_path, new_file_path)
            
            # Increment the counter
            counter += 1

# Example usage
folder_path = 'D:/PATENT/s'  # Replace with your folder path
prefix = 's'  # Replace with 'i', 's', or 'b' as needed
label_images(folder_path, prefix)


In [ ]:
# MASK SEGMENTATION

In [6]:
import os
import cv2
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from tensorflow.keras.models import Model

def unet_model(input_size=(256, 256, 1)):
    inputs = Input(input_size)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = MaxPooling2D((2, 2))(c4)

    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    u6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Initialize and compile the U-Net model
model = unet_model(input_size=(256, 256, 1))
model.summary()

# Load pre-trained weights if available
# model.load_weights('path_to_pretrained_weights.h5')

def create_masks_with_labels(image_dir, mask_dir, model):
    os.makedirs(mask_dir, exist_ok=True)

    for filename in os.listdir(image_dir):
        if filename.endswith('.png') or filename.endswith('.jpg') or filename.endswith('.jpeg'):
            label = filename.split('_')[0]  # Extract the label from the filename
            image_path = os.path.join(image_dir, filename)
            image = Image.open(image_path).convert('L')  # Convert to grayscale
            image = image.resize((256, 256))
            image_np = np.array(image) / 255.0
            image_np = np.expand_dims(image_np, axis=-1)
            image_np = np.expand_dims(image_np, axis=0)

            # Predict the mask using U-Net model
            mask = model.predict(image_np)[0, :, :, 0]
            mask = (mask > 0.5).astype(np.uint8)  # Threshold to binary mask

            # Save the mask with the same label
            mask_filename = f"{label}_{filename}"
            mask_path = os.path.join(mask_dir, mask_filename)
            cv2.imwrite(mask_path, mask * 255)

# Example usage
image_dir = 'D:/PATENT/DATASET (u-net)/images'  # Replace with the path to your images folder
mask_dir = 'D:/PATENT/DATASET (u-net)/masks'  # Replace with the path to save the masks
create_masks_with_labels(image_dir, mask_dir, model)



TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates